In [4]:
# Created by Davide Pasca - 2023/07/25

import requests
import json
import time
import datetime

def get_klines_req(symbol, interval, start_time, limit=500):
    url = 'https://api.binance.com/api/v3/klines'
    
    # Define the parameters
    params = {
        'symbol': symbol,
        'interval': interval,
        'startTime': start_time,
        'limit': limit
    }

    response = requests.get(url, params=params)
    data = json.loads(response.text)

    # Convert the data to a more readable format
    readable_data = []
    for candle in data:
        time_stamp = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(candle[0]/1000))
        ohlcv = candle[1:6]
        readable_candle = [time_stamp] + ohlcv
        readable_data.append(readable_candle)

    return readable_data

def get_klines(symbol, interval, start_date, end_date):
    # Convert the start and end dates to milliseconds
    start_time = int(start_date.timestamp() * 1000)
    end_time = int(end_date.timestamp() * 1000)

    # Fetch historical data for Bitcoin with 6 hours candles
    btc_data = []
    while True:
        new_data = get_klines_req(symbol, interval, start_time)
        if not new_data:
            break
        last_time = new_data[-1][0]
        last_time_dt = datetime.datetime.strptime(last_time, '%Y-%m-%d %H:%M:%S')
        last_time_millis = int(last_time_dt.timestamp() * 1000)
        if last_time_millis >= end_time:
            # Clamp candles that go beyond the end date
            new_data = [candle for candle in new_data if int(datetime.datetime.strptime(candle[0], '%Y-%m-%d %H:%M:%S').timestamp() * 1000) < end_time]
        btc_data += new_data
        if last_time_millis >= end_time:
            break
        start_time = last_time_millis + 1
        time.sleep(0.1)  # delay to avoid hitting rate limits

    return btc_data

In [6]:
# Define the start and end dates
start_date = datetime.datetime(2021, 1, 1)
end_date = datetime.datetime(2022, 1, 1)

def get_closing_prices(data):
    # The closing price is the 5th element in each candle
    return [float(candle[4]) for candle in data]

# Fetch data
btc_data = get_klines('BTCUSDT', '6h', start_date, end_date)

# Extract and print the closing prices
closing_prices = get_closing_prices(btc_data)
for price in closing_prices:
    print(price)

28923.63
29187.01
29313.49
29079.64
29331.69
29589.99
29755.0
32856.96
32178.33
33758.67
34413.53
32909.27
33000.05
32813.01
30743.06
31024.17
31988.71
30436.25
31776.66
32742.73
33949.53
34978.91
34918.53
34898.52
36769.36
37347.2
37822.09
39732.92
39432.28
37975.25
41286.67
41009.07
40582.81
39295.37
40565.77
40421.4
40088.22
40451.84
39181.76
38571.86
38150.02
34904.73
34198.56
32839.87
35404.47
35882.68
35410.99
35012.53
34051.24
33321.75
34385.48
34821.48
37371.38
37529.11
38365.86
39429.81
39144.5
37808.9
38108.88
35947.31
36742.22
35854.37
37680.0
36860.97
35994.98
35547.13
35267.17
35837.22
35828.61
35344.5
36442.08
35675.28
36631.27
36251.18
37266.49
37162.4
35891.49
35477.18
34426.17
34849.99
35468.23
34562.7
32208.62
31912.38
30850.13
31800.93
31710.88
32329.25
32945.17
33350.6
31645.49
32239.09
32078.0
32871.98
32833.37
31851.6
32259.9
33331.13
33185.26
33784.36
32254.2
32062.32
31796.95
32184.36
32467.77
31503.41
30639.05
30514.34
30366.15
31729.16
31437.37
32281.49
33364.